# Problem Set 3: Merging and regular expressions

**Total points (without extra credit)**: 44

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


## Resources from class

- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/06_qss20_w23_mergereshape.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/03_reshaping_merging_solutions.ipynb) on exact merging
- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/07_qss20_w23_regex.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/04_regex_blank.ipynb) on regular expressions (both coming in class on Feb. 1)
- DataCamp modules on both of these

# 0. Load packages & data (1 point total)

In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 0.1 Load data (0 points)

Load the following dataset stored in `pset3_inputdata`&mdash;`debar.csv`&mdash;storing it as a dataframe named `debar`. This represents employers temporarily banned from hiring workers.

View the `head()` and columns of this dataframe.

In [2]:
## your code here

debar = pd.read_csv('pset3_inputdata/debar.csv', low_memory= False)

debar.head()

Name     City, State  \
0                    J&J Harvesting       Leads, ND   
1            Stahlman Apiaries, Inc       Selby, SD   
2                     Trust Nursery     Pulaski, NY   
3             Anton Fertilizer Inc.     Dighton, KS   
4  Great Plains Fluid Service, Inc.  Greensburg, KS   

                                        Violation Duration Start date  \
0  Failure to respond to audit (partial response)  2 years  1/19/2014   
1  Failure to respond to audit (partial response)   1 year  2/19/2015   
2  Failure to respond to audit (partial response)   1 year  3/21/2014   
3       Failure to respond to audit (no response)  2 years  3/30/2014   
4       Failure to respond to audit (no response)  2 years  3/30/2014   

    End date  
0  1/18/2016  
1  2/14/2016  
2  3/20/2015  
3  3/29/2016  
4  3/29/2016

The most notable column names mean as follows:
- `Name`:	Company name of agricultural employer
- `City, State`:	City and state where employer located
- `Violation`:	Type of program violation
- `Start date`:	Start date of debarment (temporary ban)

## 0.2 Inspect data (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [3]:
## your code here

len(debar) #Length of debar
debar.Name.nunique() #Unique employer names

#There are multiple names for some employers, as the number of unique employer names does not add up to rows in the dataset.

114

98

# 1. Reshape data and check duplicates (19 points total)

## 1.1 Make indicator for violation number (2 points)

To make it possible to reshape data, make an indicator for the violation number for each business. The indicator should take the value of `viol` if it's the first row/potential violation, `viol2` if the second row/potential violation, etc.

**Hint:**
- One way to do this is by using an if-else statement to check whether the business name is the same as in the row above (assuming rows are ordered by name). Grouping by employer name and checking the number of unique offenses would also work.

In [4]:
## your code here
debar = debar.sort_values("Name").reset_index(drop = True)

debar['viol_num'] = ["viol2" if debar["Name"].iloc[i] == debar["Name"].iloc[i-1] else "viol1" for i in debar.index]

debar.sample(n = 100)

Name      City, State  \
45                   J & L Farms    Claremont, SD   
106         Valley View Orchards              NaN   
78         Peach Crest Farm, LLC    Stratford, OK   
95                 Sharon Mathis       Tifton, GA   
47           Javier Xisto Zapata   Lyons, Georgia   
..                           ...              ...   
112               Yolanda Chavez  Santa Maria, CA   
77               Omega Lamb, LLC    El Centro, CA   
105                 Turner Farms        Healy, KS   
88   Rollo Farm Labor Contractor        Miami, FL   
97          Slash E.V. Ranch LLP        Rifle, CO   

                                             Violation  Duration  Start date  \
45                                        WHD referral    1 year    5/8/2018   
106     Impeding the Audit Process – Partial Response;   2 years   7/22/2015   
78                                 Wage Hour Debarment   3 years   2/10/2015   
95           Failure to respond to audit (no response)   2 years  11/16/2014   
47                    Non-payment of certification fee    1 year  10/22/2016   
..                                                 ...       ...         ...   
112                                        Non Payment    1 year   7/23/2014   
77                                         Non-payment    1 year    3/4/2016   
105  Failure to comply with the employer's obligati...  7 months     7/17/19   
88           Failure to respond to audit (no response)   2 years   8/23/2014   
97                                       WHD Debarment    1 year  11/15/2014   

       End date viol_num  
45     5/7/2019    viol1  
106   7/21/2017    viol1  
78     2/8/2018    viol1  
95   11/15/2016    viol1  
47   10/21/2017    viol1  
..          ...      ...  
112   7/22/2015    viol1  
77     3/3/2017    viol1  
105     2/10/20    viol2  
88    8/22/2016    viol1  
97   11/14/2015    viol1  

[100 rows x 7 columns]

## 1.2 Clean up state names (3 points)

Inspect the state names in the business violation data. Which states have names sometimes indicated in long format vs. two-letter abbreviation, e.g. "New Hampshire" vs. "NH"? Which of these may have more than one violation?

**Hint:** 
- One way to do this is to extract state names from the `City, State` column using regex and/or string methods.
- The simplest way to check if a state may have more than one violation is to check how many times they appear using `value_counts()`. 

In [8]:
from collections import Counter


In [28]:

## States with names indicated in long format vs. two-letter abbreviation

#Listing all the states in both full and abbreviated names
states = []

for x in debar['City, State']:
    abbrev = re.findall(r'\b[A-Z]{2}\b|\b\w+\b,\s(?:\w+\b)', str(x))
    if abbrev:
        if len(abbrev[0]) == 2:
            states.append(abbrev[0])
        else:
            split_match = abbrev[0].split(',')[-1].strip().split()
            if len(split_match) >= 2 and split_match[0] == "North":
                states.append(f"North Dakota")
            else:
                states.append(split_match[-1])
    else:
        states.append('')

for i, state in enumerate(states):
    if state == "North":
        states[i] = "North Dakota"
        
len(states)
states

#Shows 114 entries




114

['ID',
 'TX',
 'TX',
 'OK',
 'SD',
 'ND',
 'FL',
 'UT',
 'UT',
 'KS',
 'MA',
 'MA',
 'LA',
 'GA',
 'MT',
 'TX',
 'TX',
 'KY',
 'GA',
 'GA',
 'GA',
 'GA',
 'ME',
 'LA',
 'GA',
 'MA',
 'NC',
 'GA',
 'SC',
 'GA',
 'TX',
 'TX',
 'CO',
 'GA',
 'KS',
 'KS',
 'TX',
 'TX',
 'ND',
 'UT',
 'AR',
 'CA',
 'KS',
 'FL',
 'KS',
 'SD',
 'ND',
 'GA',
 'MN',
 'FL',
 'TN',
 'TX',
 'KY',
 'NY',
 'CO',
 'MT',
 'MD',
 'LA',
 'KY',
 'GA',
 'TX',
 'TX',
 'ID',
 'NY',
 'TX',
 'TX',
 'AR',
 'AR',
 'UT',
 'UT',
 'AR',
 'GA',
 'GA',
 'GA',
 'SD',
 'SD',
 'ND',
 'CA',
 'OK',
 'TX',
 'KS',
 'ND',
 'FL',
 'FL',
 'FL',
 'KS',
 'KS',
 'KY',
 'FL',
 'FL',
 'FL',
 'FL',
 'TX',
 'TX',
 'VT',
 'GA',
 'GA',
 'CO',
 'SD',
 'KY',
 'ND',
 'TX',
 'TX',
 'NY',
 'KS',
 'KS',
 '',
 'UT',
 'IL',
 'GA',
 'GA',
 'GA',
 'CA',
 'CA']

In [13]:
#Listing the number of times each have violations

state_counts = Counter(states)

state_counts


Counter({'Idaho': 1,
         'Texas': 4,
         'TX': 14,
         'OK': 2,
         'SD': 5,
         'ND': 3,
         'FL': 8,
         'Utah': 2,
         'UT': 4,
         'KS': 9,
         'MA': 2,
         'Louisiana': 1,
         'Georgia': 9,
         'MT': 2,
         'KY': 5,
         'GA': 10,
         'ME': 1,
         'LA': 2,
         'Massachusetts': 1,
         'North Dakota': 4,
         'SC': 1,
         'CO': 3,
         'AR': 2,
         'California': 1,
         'MN': 1,
         'Tennessee': 1,
         'NY': 3,
         'MD': 1,
         'ID': 1,
         'AK': 1,
         'Arkansas': 1,
         'CA': 3,
         'Florida': 2,
         'Kansas': 1,
         'Vermont': 1,
         '': 1,
         'IL': 1})

To make the state names consistent with the jobs data below, convert any discrepant state names to the two-letter abbreviation format. 

**Hints:**
- You could do this for all state names (to be safe) or only those you just identified. 
- If you want to change ANY discrepant state names from long format to two-letter format (i.e., a complete conversion), you can load state names from a complete name/abbreviation crosswalk and use that to change state names. Here is code to load in such a list ([from this blog](https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c)):
```python
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
pd.read_html(crosswalk_url)[0]
```

In [10]:
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
crosswalk = pd.read_html(crosswalk_url)[0]

In [11]:
## your code here
#Help from GeeksforGeeks

def abbreviate_state(row):
    city_state = row["City, State"]
    pattern = r"(.*),\s*([A-Za-z ]+)"
    match = re.search(pattern, city_state)
    if match:
        city, state = match.group(1), match.group(2)
        state_abbrev = crosswalk[crosswalk['Description'] == state]['Code'].values
        if state_abbrev.size > 0:
            state_abbrev = state_abbrev[0]
        else:
            state_abbrev = state
        return f"{city}, {state_abbrev}"
    return city_state

debar["City, State"] = debar["City, State"].astype(str)

debar["City, State"] = debar.apply(abbreviate_state, axis=1)
debar["City, State"]

0           Preston, ID
1      Stephenville, TX
2          Seminole, TX
3            Idabel, OK
4           Ipswich, SD
             ...       
109           Lyons, GA
110           Lyons, GA
111           Axson, GA
112     Santa Maria, CA
113     Santa Maria, CA
Name: City, State, Length: 114, dtype: object

In [173]:
debar["City, State"].head()

0         Preston, ID
1    Stephenville, TX
2        Seminole, TX
3          Idabel, OK
4         Ipswich, SD
Name: City, State, dtype: object

## 1.3 Investigate duplicated rows (4 points)

A. Create a new column in `debar`, `is_repeated`, that tells us whether an employer (`Name`) is repeated > 1 times.

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using `value_counts()` on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret with at least a sentence. If you notice any cases of duplicate business names where 'City, State' does NOT match exactly, check these manually and make them consistent.

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [14]:
debar['City, State'] = np.where(debar['City, State'] == 'Brownsville, TX', 'Brownfield, TX', debar['City, State'])
debar['City, State'] = np.where(debar['City, State'] == 'Altheimer, AK', 'Altheimer, AR', debar['City, State'])

In [19]:
## your code here

#Part A

duplicates = debar[debar.duplicated(['Name'])]['Name'].tolist()
debar['is_repeated'] = [False if debar["Name"].iloc[name] not in duplicates else True for name in debar.index]

#Part B

print(debar[debar['is_repeated']==True])

#Ther are no cases where the City and State does not match the previous row.
#There are 32 observations so that means that there are 16 names that are repeated.

#Part C

mult_debar= debar[debar['is_repeated']==True]

mult_debar.head()
mult_debar.shape

                                        Name       City, State  \
7                    Annabella Land & Cattle     Annabella, UT   
8                    Annabella Land & Cattle     Annabella, UT   
10                       Autumn Hill Orchard        Groton, MA   
11                       Autumn Hill Orchard        Groton, MA   
15     Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
16     Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
19                        Cisco Produce Inc.         Cairo, GA   
20                        Cisco Produce Inc.         Cairo, GA   
30                          Dove Creek Farms  Mount Vernon, TX   
31                          Dove Creek Farms  Mount Vernon, TX   
34                                 F&W Farms       Ingalls, KS   
35                                 F&W Farms       Ingalls, KS   
60                     Loewen Harvesting LLC    Brownfield, TX   
61                     Loewen Harvesting LLC    Brownfield, TX   
64        

Name    City, State  \
7                 Annabella Land & Cattle  Annabella, UT   
8                 Annabella Land & Cattle  Annabella, UT   
10                    Autumn Hill Orchard     Groton, MA   
11                    Autumn Hill Orchard     Groton, MA   
15  Caddo Creek Ranch, dba Paradise Ranch      Caddo, TX   

                                         Violation Duration Start date  \
7                                      Non-payment   1 year   5/9/2014   
8                                      Non Payment   1 year   5/9/2014   
10       Failure to respond to audit (no response)  2 years   7/6/2014   
11      Impeding the Audit Process – Non- Response  2 years   7/6/2014   
15  Impeding the Audit Process – Partial- Response  2 years  7/20/2014   

     End date viol_num  is_repeated  
7    5/8/2015    viol1         True  
8    5/9/2015    viol2         True  
10   7/5/2016    viol1         True  
11   7/5/2016    viol2         True  
15  7/19/2016    viol1         True

(32, 8)

## 1.4 Reshape `mult_debar` to wide format to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in these data: e.g., converting `Violation` to lowercase or replacing spelled-out states with two-dig state codes.

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

For a successful reshaping, make sure each row in `mult_debar` shows only a single business (no duplicates under `Name`) and isn't missing either `start_date_viol1` or `start_date_viol2`.

Show the contents of `mult_debar` and its shape.

In [16]:
## your code here

mult_debar_wide = pd.pivot(mult_debar, index=['Name', 'City, State'], columns= 'viol_num', values ='Start date')

mult_debar_wide.columns= ["start_date_viol1", "start_date_viol2"]


mult_debar_wide.head()
mult_debar_wide.shape
mult_debar_wide

start_date_viol1  \
Name                                  City, State                         
Annabella Land & Cattle               Annabella, UT            5/9/2014   
Autumn Hill Orchard                   Groton, MA               7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX               7/20/2014   
Cisco Produce Inc.                    Cairo, GA              12/10/2014   
Dove Creek Farms                      Mount Vernon, TX         2/9/2018   

                                                       start_date_viol2  
Name                                  City, State                        
Annabella Land & Cattle               Annabella, UT            5/9/2014  
Autumn Hill Orchard                   Groton, MA               7/6/2014  
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX               7/20/2014  
Cisco Produce Inc.                    Cairo, GA              12/10/2015  
Dove Creek Farms                      Mount Vernon, TX         2/9/2018

(16, 2)

start_date_viol1  \
Name                                    City, State                         
Annabella Land & Cattle                 Annabella, UT            5/9/2014   
Autumn Hill Orchard                     Groton, MA               7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch   Caddo, TX               7/20/2014   
Cisco Produce Inc.                      Cairo, GA              12/10/2014   
Dove Creek Farms                        Mount Vernon, TX         2/9/2018   
F&W Farms                               Ingalls, KS            12/10/2014   
Loewen Harvesting LLC                   Brownfield, TX          8/20/2014   
Macky and Brad Farms                    Plains, TX              2/13/2015   
Maple Ridge Custom Services, LLC        Altheimer, AR          11/16/2014   
Mark Duncan                             Roosevelt, UT          11/16/2014   
Old Tree Farms/Verpaalen Custom Service Volga, SD              12/11/2014   
Rollo Farm Labor Contractor             Miami, FL               8/23/2014   
SRT Farms                               Morton, TX             11/16/2014   
Sharon Mathis                           Tifton, GA             11/16/2014   
Turner Farms                            Healy, KS                 7/17/19   
Xavier Horne                            Lyons, GA               6/16/2016   

                                                         start_date_viol2  
Name                                    City, State                        
Annabella Land & Cattle                 Annabella, UT            5/9/2014  
Autumn Hill Orchard                     Groton, MA               7/6/2014  
Caddo Creek Ranch, dba Paradise Ranch   Caddo, TX               7/20/2014  
Cisco Produce Inc.                      Cairo, GA              12/10/2015  
Dove Creek Farms                        Mount Vernon, TX         2/9/2018  
F&W Farms                               Ingalls, KS            12/10/2014  
Loewen Harvesting LLC                   Brownfield, TX          8/20/2014  
Macky and Brad Farms                    Plains, TX              2/13/2015  
Maple Ridge Custom Services, LLC        Altheimer, AR          11/16/2014  
Mark Duncan                             Roosevelt, UT          11/16/2014  
Old Tree Farms/Verpaalen Custom Service Volga, SD               12/1/2014  
Rollo Farm Labor Contractor             Miami, FL               8/23/2014  
SRT Farms                               Morton, TX             11/16/2014  
Sharon Mathis                           Tifton, GA             11/16/2014  
Turner Farms                            Healy, KS                 7/17/19  
Xavier Horne                            Lyons, GA               9/27/2017

## 1.5 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning, `debar`, filter as follows:
- For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
- For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
- Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [18]:
## your code here

mult_debar_wide['is_dup'] = np.equal(mult_debar_wide['start_date_viol1'], mult_debar_wide['start_date_viol2'])

mult_debar_wide = mult_debar_wide.reset_index()

dups = mult_debar_wide["Name"][mult_debar_wide["is_dup"] == True]

# mult_debar_wide

debar_clean = debar[(debar['Name'].isin(dups) & debar['viol_num'] == "viol1") | ~debar['Name'].isin(dups)] 

# # not_dup = debar[~debar['Name'].isin(is_dup_list)]

debar_clean = debar_clean.drop(columns = 'is_repeated')
debar_clean = debar_clean.reset_index(drop=True)

debar_clean


Name       City, State  \
0             69 Farms LLC       Preston, ID   
1                 AB Ranch  Stephenville, TX   
2        Abe- Nancy Froese      Seminole, TX   
3              Agecy I LLC        Idabel, OK   
4            Agecy II, LLC       Ipswich, SD   
..                     ...               ...   
83            Xavier Horne         Lyons, GA   
84            Xavier Horne         Lyons, GA   
85           Yesenia Perez         Axson, GA   
86          Yolanda Chavez   Santa Maria, CA   
87  Yolanda Chavez Farming   Santa Maria, CA   

                                     Violation Duration  Start date  \
0          Failure to respond to audit request  2 years   9/24/2016   
1          Failure to respond to audit request  2 years   4/28/2017   
2          Failure to respond to audit request  2 years  11/23/2017   
3   Impeding the Audit Process – Non- Response  2 years   8/27/2015   
4   Impeding the Audit Process – Non- Response  2 years   3/25/2015   
..                                         ...      ...         ...   
83            Non-payment of certification fee   1 year   6/16/2016   
84         Failure to respond to audit request  2 years   9/27/2017   
85         Failure to respond to audit request  2 years   9/27/2017   
86                                 Non Payment   1 year   7/23/2014   
87                                 Non-Payment   1 year   7/23/2014   

      End date viol_num  
0    9/23/2018    viol1  
1    4/27/2019    viol1  
2   11/23/2019    viol1  
3    8/26/2017    viol1  
4    3/24/2017    viol1  
..         ...      ...  
83   6/15/2017    viol1  
84   9/26/2019    viol2  
85   9/26/2019    viol1  
86   7/22/2015    viol1  
87   7/22/2015    viol1  

[88 rows x 7 columns]

# 2. Merging and regex (19 points total)



## 2.1 Load data on job postings (1 point)

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations. Since most of the bans have expired, let's see which of those employers posted new H-2A jobs in the first quarter of 2021.

First, load the `jobs.csv` data stored in `pset3_inputdata`.

In [197]:
# your code here to load the data 

jobs = pd.read_csv('pset3_inputdata/jobs.csv', low_memory= False)

jobs.head()

CASE_NUMBER                           CASE_STATUS  \
0  H-300-20199-721302      Determination Issued - Withdrawn   
1  H-300-20231-773906  Determination Issued - Certification   
2  H-300-20231-774123  Determination Issued - Certification   
3  H-300-20231-774151  Determination Issued - Certification   
4  H-300-20231-774508  Determination Issued - Certification   

             RECEIVED_DATE            DECISION_DATE  \
0  2020-07-17 14:50:40.840  2020-10-01 00:00:00.000   
1  2020-08-20 10:38:15.620  2020-10-01 00:00:00.000   
2  2020-08-24 15:33:14.340  2020-10-01 00:00:00.000   
3  2020-08-21 12:08:09.760  2020-10-01 00:00:00.000   
4  2020-08-20 10:17:34.530  2020-10-01 00:00:00.000   

  TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR NATURE_OF_TEMPORARY_NEED  \
0          Individual Employer                    N                 Seasonal   
1          Association - Agent                    N                 Seasonal   
2          Individual Employer                    N                 Seasonal   
3          Individual Employer                    N                 Seasonal   
4          Individual Employer                    Y                 Seasonal   

  EMERGENCY_FILING                       EMPLOYER_NAME  \
0                Y  Fazio Farms Operating Company, LLC   
1                N                  Charlie Sunderland   
2                N                   Michael Rudebusch   
3                N                        Lodahl Farms   
4                N             Dunson Harvesting, Inc.   

                TRADE_NAME_DBA  ... ADDENDUM_B_HOUSING_ATTACHED  \
0                          NaN  ...                           N   
1  Panter & Sunderland Nursery  ...                           N   
2                          NaN  ...                           N   
3                          NaN  ...                           Y   
4      Dunson Harvesting, Inc.  ...                           Y   

  TOTAL_HOUSING_RECORDS MEALS_PROVIDED MEALS_CHARGED  \
0                     1              Y         12.68   
1                     1              N           NaN   
2                     1              N           NaN   
3                     2              N           NaN   
4                     8              N           NaN   

  MEAL_REIMBURSEMENT_MINIMUM MEAL_REIMBURSEMENT_MAXIMUM PHONE_TO_APPLY  \
0                      12.68                       55.0    13607017661   
1                      12.68                       55.0    19318083783   
2                      12.68                       55.0    19369333827   
3                      12.68                       55.0    14069637560   
4                      12.68                       55.0    18632939888   

                  EMAIL_TO_APPLY                             WEBSITE_TO_APPLY  \
0           faziofarms@gmail.com                                          NaN   
1                            NaN  https://www.jobs4tn.gov/vosnet/Default.aspx   
2  fayethlynpitre@rocketmail.com                                          NaN   
3        lodahl_kelsey@yahoo.com                                          NaN   
4                            NaN                        www.employflorida.com   

   TOTAL_ADDENDUM_A_RECORDS  
0                         0  
1                         0  
2                         0  
3                         0  
4                         4  

[5 rows x 138 columns]

The most notable column names mean as follows:
- `CASE_NUMBER`:	Administrative identifier for an employer's H-2A visa application
- `EMPLOYER_NAME`:	Employer name
- `EMPLOYER_CITY`:	Employer city
- `EMPLOYER_STATE`:	Employer state
- `EMPLOYER_ADDRESS_1`:	Employer address (only need to use if doing the geocoding extra credit)

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [198]:
## your code here

#A

df_merged = pd.merge(jobs, debar_clean, left_on='EMPLOYER_NAME', right_on='Name', how='inner')

#B

df_merged

#1 exact match, as the dataframe shape is (1 x 145)

CASE_NUMBER                           CASE_STATUS  \
0  H-300-20287-876656  Determination Issued - Certification   

             RECEIVED_DATE            DECISION_DATE  \
0  2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   

  TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR NATURE_OF_TEMPORARY_NEED  \
0          Individual Employer                    Y                 Seasonal   

  EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  ... EMAIL_TO_APPLY  \
0                Y  Rafael Barajas            NaN  ...            NaN   

                WEBSITE_TO_APPLY TOTAL_ADDENDUM_A_RECORDS            Name  \
0  https://seasonaljobs.dol.gov/                        7  Rafael Barajas   

   City, State                         Violation Duration  Start date  \
0  Sebring, FL  Non-payment of certification fee   1 year   9/23/2016   

    End date  viol_num  
0  9/22/2017     viol1  

[1 rows x 145 columns]

## 2.3 Targeted regex (11 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Convert to upper (3 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [199]:
## insert your code to turn into uppercase here

#A
jobs["EMPLOYER_NAME"] = [name.upper() for name in jobs["EMPLOYER_NAME"]]

debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]

In [200]:
## insert your code for the random sample

#B

jobs["EMPLOYER_NAME"].sample(n=15)
debar_clean["Name"].sample(n=15)

2409      BAYOU BEST CRAWFISH, LLC
898            FRESH HARVEST, INC.
1168     WESTERN RANGE ASSOCIATION
1189           LOUIS SCATENA RANCH
2459      PRAIRIE RIDGE FARMS INC.
2279    NORMAN KEIL NURSERIES INC.
958      WESTERN RANGE ASSOCIATION
783        BOMAR AGRA ESTATES, LLC
431                HBW FARMS, INC.
24       WESTERN RANGE ASSOCIATION
559       DUNCAN FAMILY FARMS LLC 
268           P&L HARVESTING, LLC 
556                 WADE FARMS INC
2446               GERLACH FARMING
156        GREEN EMPIRE FARMS, INC
Name: EMPLOYER_NAME, dtype: object

62                         PROMAX INC.
26              FIRST AMERICAN HOLDING
29                  GERONIMO SHEEP CO.
4                        AGECY II, LLC
14                  CISCO PRODUCE INC.
6                 ALTERIC JEAN-CHARLES
73                SLASH E.V. RANCH LLP
20                    DEAL FAMILY FARM
25                   EVERGREEN PRODUCE
45                       LABATTE FARMS
51                     M KENESTON CORP
23                         DONNA LUCIO
33            HAITI/USA WORKFORCE, LLC
44    LOV RANCH CO, LLLP DBA LOV RANCH
30                 GLENDA HARROD FARMS
Name: Name, dtype: object

In [201]:
## insert your code for assigning the uppercase names back to the data

#C
jobs["EMPLOYER_NAME"] = [name.upper() for name in jobs["EMPLOYER_NAME"]]

debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]


### 2.3.2 Clean up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [202]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [203]:
## insert your code here with the regex pattern for part A
pattern = r'(INC|LLC|CO)\.' #For both

## insert your code to use re.sub to apply the pattern to the test cases for part B
print(re.sub(pattern, r'\1', pos_example_1))
print(re.sub(pattern, r'\1', pos_example_2))
print(re.sub(pattern, r'\1', neg_example))

CISCO PRODUCE INC
AVOYELLES HONEY CO, LLC
E.V. RANCH LLP


### 2.3.3 Clean employer names (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [204]:
## your code here to clean the columns
debar_clean["name_clean"] = [re.sub(r"(INC|LLC|CO)\.", r"\1", name) for name in debar_clean["Name"]]
jobs["name_clean"] = [re.sub(r"(INC|LLC|CO)\.", r"\1", name) for name in jobs["EMPLOYER_NAME"]]

In [205]:
## your code here to print the head
debar_clean["name_clean"].head()
jobs["name_clean"].head()

0         69 FARMS LLC
1             AB RANCH
2    ABE- NANCY FROESE
3          AGECY I LLC
4        AGECY II, LLC
Name: name_clean, dtype: object

0    FAZIO FARMS OPERATING COMPANY, LLC
1                    CHARLIE SUNDERLAND
2                     MICHAEL RUDEBUSCH
3                          LODAHL FARMS
4                DUNSON HARVESTING, INC
Name: name_clean, dtype: object

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe. Write a sentence describing how name cleaning affected the match results.

**Note**: this manual cleaning process is inefficient and may miss other likely matches. A better approach would be fuzzy matching, which would recognize that 'Slash EV ranch' is a highly similar string to 'slash ev ranch llp' and match them without us needing to use regex to make the strings identical. We may talk about this in class later if time!

In [206]:
## your code here

#A
df_merge2 = pd.merge(jobs, debar_clean, left_on ='name_clean', right_on ='name_clean', how ='inner')

print(df_merge2)

          CASE_NUMBER                           CASE_STATUS  \
0  H-300-20287-876656  Determination Issued - Certification   

             RECEIVED_DATE            DECISION_DATE  \
0  2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   

  TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR NATURE_OF_TEMPORARY_NEED  \
0          Individual Employer                    Y                 Seasonal   

  EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  ...  \
0                Y  RAFAEL BARAJAS            NaN  ...   

                WEBSITE_TO_APPLY TOTAL_ADDENDUM_A_RECORDS      name_clean  \
0  https://seasonaljobs.dol.gov/                        7  RAFAEL BARAJAS   

             Name  City, State                         Violation Duration  \
0  RAFAEL BARAJAS  Sebring, FL  Non-payment of certification fee   1 year   

   Start date   End date  viol_num  
0   9/23/2016  9/22/2017     viol1  

[1 rows x 146 columns]


In [207]:
#B
debar_clean['name_clean_2'] = debar_clean['Name'].str.replace("\.", "").str.replace(" LLP", "")
debar_clean['name_clean_2']

/var/folders/yc/cnm4b7xx2z7frdrr6dvcz2180000gn/T/ipykernel_18150/3733005839.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  debar_clean['name_clean_2'] = debar_clean['Name'].str.replace("\.", "").str.replace(" LLP", "")


0               69 FARMS LLC
1                   AB RANCH
2          ABE- NANCY FROESE
3                AGECY I LLC
4              AGECY II, LLC
               ...          
83              XAVIER HORNE
84              XAVIER HORNE
85             YESENIA PEREZ
86            YOLANDA CHAVEZ
87    YOLANDA CHAVEZ FARMING
Name: name_clean_2, Length: 88, dtype: object

In [208]:
#C

df_merge3 = pd.merge(jobs, debar_clean, how = "left", left_on = "name_clean", right_on="name_clean_2", indicator = True)

print(df_merge3[df_merge3["_merge"] == "both"])

             CASE_NUMBER                           CASE_STATUS  \
791   H-300-20287-876656  Determination Issued - Certification   
1115  H-300-20306-894148  Determination Issued - Certification   

                RECEIVED_DATE            DECISION_DATE  \
791   2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   
1115  2020-11-02 18:11:29.140  2020-11-24 00:00:00.000   

     TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR  \
791           Individual Employer                    Y   
1115          Individual Employer                    N   

     NATURE_OF_TEMPORARY_NEED EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  \
791                  Seasonal                Y  RAFAEL BARAJAS            NaN   
1115                 Seasonal                N  SLASH EV RANCH            NaN   

      ...                  Name  City, State  \
791   ...        RAFAEL BARAJAS  Sebring, FL   
1115  ...  SLASH E.V. RANCH LLP    Rifle, CO   

                             Violation Duration  Start date   

The name cleaning increased the match results by 1 with a left join, indicating that we were able to change the names enough so that they were recognized as the same. 

# 3. Regex to separate companies from individuals (6 points)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [24]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
pattern3 = r'(.*?) \((COMPANY|INDIVIDUAL)\)'

## your code here to apply it to the pos_example
comp_ind = re.findall(pattern3, pos_example)

#With help from Stackoverflow
if comp_ind:
    company = comp_ind[0][0].strip() if comp_ind[0][1] == 'COMPANY' else ''
    individual = comp_ind[1][0].strip().replace("AND ", "") if comp_ind[1][1] == 'INDIVIDUAL' else ''
    print('Company Name:', company)
    print('Individual Name:', individual)

else:
    print('None')
    
## your code here to apply it to the negative example
comp_ind = re.findall(pattern3, neg_example)

if comp_ind:
    company = comp_ind[0][0].strip() if comp_ind[0][1] == 'COMPANY' else ''
    individual = comp_ind[1][0].strip().replace("AND ", "") if comp_ind[1][1] == 'INDIVIDUAL' else ''
    print('Company Name:', company)
    print('Individual Name:', individual)

else:
    print('None')


Company Name: COUNTY FAIR FARM
Individual Name: ANDREW WILLIAMSON
None


C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [261]:
# your code here

comp_ind_pattern = r'(.*?) \((COMPANY|INDIVIDUAL)\)'

companies = []
individuals = []

#With help from Stackoverflow

for name in debar_clean['Name']:
    comp_ind = re.findall(comp_ind_pattern, name)
    company= name
    individual= name
    if len(comp_ind)>= 1:
        company = comp_ind[0][0].strip() if comp_ind[0][1] == 'COMPANY' else ''
        if len(comp_ind)>= 2:
            individual = comp_ind[1][0].strip().replace("AND ", "") if comp_ind[1][1] == 'INDIVIDUAL' else ''
            
    companies.append(company)
    individuals.append(individual)

debar_clean['co_name'] = companies
debar_clean['ind_name'] = individuals

debar_clean[['name_clean', "co_name", "ind_name"]].head(17) #See index 17

name_clean  \
0                                        69 FARMS LLC   
1                                            AB RANCH   
2                                   ABE- NANCY FROESE   
3                                         AGECY I LLC   
4                                       AGECY II, LLC   
5                             ALTENDORF TRANSPORT INC   
6                                ALTERIC JEAN-CHARLES   
7                                ANTON FERTILIZER INC   
8                             AVOYELLES HONEY CO, LLC   
9         B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)   
10                              C HANGING L RANCH LLP   
11                                     CHRIS ROBINSON   
12                                  CIRA CORTEZ LOPEZ   
13                                  CISCO PRODUCE INC   
14                                  CISCO PRODUCE INC   
15                      CITY PINESTRAW AND HARVESTING   
16  COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...   

                          co_name                                     ind_name  
0                    69 FARMS LLC                                 69 FARMS LLC  
1                        AB RANCH                                     AB RANCH  
2               ABE- NANCY FROESE                            ABE- NANCY FROESE  
3                     AGECY I LLC                                  AGECY I LLC  
4                   AGECY II, LLC                                AGECY II, LLC  
5        ALTENDORF TRANSPORT INC.                     ALTENDORF TRANSPORT INC.  
6            ALTERIC JEAN-CHARLES                         ALTERIC JEAN-CHARLES  
7           ANTON FERTILIZER INC.                        ANTON FERTILIZER INC.  
8        AVOYELLES HONEY CO., LLC                     AVOYELLES HONEY CO., LLC  
9                                  B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)  
10          C HANGING L RANCH LLP                        C HANGING L RANCH LLP  
11                 CHRIS ROBINSON                               CHRIS ROBINSON  
12              CIRA CORTEZ LOPEZ                            CIRA CORTEZ LOPEZ  
13             CISCO PRODUCE INC.                           CISCO PRODUCE INC.  
14             CISCO PRODUCE INC.                           CISCO PRODUCE INC.  
15  CITY PINESTRAW AND HARVESTING                CITY PINESTRAW AND HARVESTING  
16               COUNTY FAIR FARM                            ANDREW WILLIAMSON

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [270]:
# your code here

#Positive
debar_clean.loc[16, ['name_clean', 'co_name', 'ind_name', 'Violation']]

#Negative
debar_clean.loc[13, ['name_clean', 'co_name', 'ind_name', 'Violation']]


name_clean    COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...
co_name                                        COUNTY FAIR FARM
ind_name                                      ANDREW WILLIAMSON
Violation                                         WHD Debarment
Name: 16, dtype: object

name_clean                            CISCO PRODUCE INC
co_name                              CISCO PRODUCE INC.
ind_name                             CISCO PRODUCE INC.
Violation     Failure to respond to audit (no response)
Name: 13, dtype: object

# 4. Optional extra credit: Geospatial visualization (2 points)

Geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on a map of Georgia. This involves Googling and using external sources to figure out the code (a common practice in real-life data science), since we haven't spatial data in the course. 

**Hints:**
- Relevant columns include `EMPLOYER_ADDRESS_1` 
- The geocoding might have a long runtime, so feel free to implement it in a separate `.py` script that you submit alongside your notebook and to just read in the geocoded data

**Resources:**
- [Discussion of geocoding addresses -> lat/long](https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/)
- [Discussion of plotting lat/long dots against a map using geopandas](https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73)

In [ ]:
## your code here